In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install git+https://github.com/modestyachts/ImageNetV2_pytorch

In [ ]:
import os
import clip
import torch
from tqdm import tqdm
from imagenetv2_pytorch import ImageNetV2Dataset

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)
dataset = ImageNetV2Dataset("matched-frequency")

In [ ]:
BATCH_SIZE = 50
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=16, shuffle=False)
ind, fin_features, fin_fnames = 0, [], []
with torch.no_grad():
  for idx, (images, target) in tqdm(enumerate(dataloader)):
    fnames = [dataset.fnames[i].as_posix() for i in range(ind, ind + BATCH_SIZE)] # batch_size of 50
    fin_fnames.append(fnames)
    images = images.to(device)
    image_features = model.encode_image(images)
    fin_features.append(image_features)
    ind += BATCH_SIZE
fin_features = torch.cat(fin_features)
print(fin_features.shape, len(fin_fnames))

In [ ]:
import pickle

torch.save(fin_features, 'imagenetv2_matched_freq_features.pt')
fnames = [f for flist in fin_fnames for f in flist]
with open("filenames.txt", "wb") as f:
  pickle.dump(fnames, f)